<a href="https://colab.research.google.com/github/caashka/programming_big_data/blob/main/%D0%94%D0%9F%D0%9E_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_Python_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_4_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практическая работа №4. Введение в Python. ООП рефакторинг**

---




**Обучающийся:** *[Введите ваше ФИО]*  



---

## **Цель работы:**



Провести рефакторинг ранее созданного Python-пакета в предыдущей практической работе, преобразовав его функциональность в соответствии с принципами объектно-ориентированного программирования. Обновленный пакет должен включать классы и методы для преобразования координат между декартовой и сферической системами координат, а также для работы с файлами.



## **Задачи:**



1. Анализ существующего пакета и определение необходимых изменений для перехода на ООП-парадигму.

2. Создание новой структуры пакета с использованием классов и модулей, соответствующих принципам ООП.

3. Реализация классов и методов для преобразования координат между декартовой и сферической системами координат.

4. Реализация классов и методов для работы с файлами, обеспечивая удобный интерфейс для чтения и записи данных.

5. Создание файла `__main__.py` с консольным интерфейсом для взаимодействия с функциональностью пакета.

6. Тестирование и проверка работоспособности обновленного пакета на примерах, подтверждение корректности реализации.



## **Демонстрация результата:**



Вставьте код каждого из ваших модулей в соответствующие ячейки ниже.

**Строку, начинающуюся на %%writefile... стирать запрещено**

### **Содержимое модуля \_\_init__.py:**

In [2]:
!mkdir geo_transform

mkdir: cannot create directory ‘geo_transform’: File exists


In [3]:
%%writefile geo_transform/__init__.py
from .transformations import CartesianCoordinates, SphericalCoordinates
from .utils import AngleConverter
from .file_operations import CoordinateFileManager
from .__main__ import show

__all__ = [
    "CartesianCoordinates",
    "SphericalCoordinates",
    "AngleConverter",
    "CoordinateFileManager",
    "show"
]

Writing geo_transform/__init__.py


### **Содержимое модуля transformations.py:**

In [4]:
%%writefile geo_transform/transformations.py
from math import sqrt, atan2, sin, cos

class CartesianCoordinates:
    def __init__(self, x=0, y=0, z=0):
        self.x = x
        self.y = y
        self.z = z

    def to_spherical(self):
        r = sqrt(self.x**2 + self.y**2 + self.z**2)
        theta = 2 * atan2(self.y, self.x + sqrt(self.x**2 + self.y**2))
        phi = atan2(sqrt(self.x**2 + self.y**2), self.z)
        return SphericalCoordinates(r, theta, phi)

    def __repr__(self):
        return f"CartesianCoordinates(x={self.x}, y={self.y}, z={self.z})"


class SphericalCoordinates:
    def __init__(self, r=0, theta=0, phi=0):
        self.r = r
        self.theta = theta
        self.phi = phi

    def to_cartesian(self):
        x = self.r * sin(self.phi) * cos(self.theta)
        y = self.r * sin(self.phi) * sin(self.theta)
        z = self.r * cos(self.phi)
        return CartesianCoordinates(x, y, z)

    def __repr__(self):
        return f"SphericalCoordinates(r={self.r}, theta={self.theta}, phi={self.phi})"

Writing geo_transform/transformations.py


### **Содержимое модуля utils.py:**

In [5]:
%%writefile geo_transform/utils.py
from math import pi

class AngleConverter:
    @staticmethod
    def deg_to_rad(deg):
        return deg * pi / 180

    @staticmethod
    def rad_to_deg(rad):
        return rad * 180 / pi

Writing geo_transform/utils.py


### **Содержимое модуля file_operations.py:**

In [6]:
%%writefile geo_transform/file_operations.py
class CoordinateFileManager:
    @staticmethod
    def write_to_file(filename, coordinates):
        with open(filename, "w", encoding="utf-8") as f:
            for line in coordinates:
                line_str = " ".join(map(str, line))
                f.write(line_str + "\n")

    @staticmethod
    def read_from_file(filename):
        with open(filename, "r", encoding="utf-8") as f:
            lines = f.read().strip().split('\n')
            return [list(map(float, line.split(' '))) for line in lines]

Writing geo_transform/file_operations.py


### **Содержимое модуля \_\_main__.py:**

In [12]:
%%writefile geo_transform/__main__.py
try:
    from .transformations import CartesianCoordinates, SphericalCoordinates
    from .utils import AngleConverter
    from .file_operations import CoordinateFileManager
except:
    from transformations import CartesianCoordinates, SphericalCoordinates
    from utils import AngleConverter
    from file_operations import CoordinateFileManager

__all__ = ['GeoTransformApp']


class GeoTransformApp:
    def __init__(self):
        self.cartesian = CartesianCoordinates()
        self.spherical = SphericalCoordinates()
        self.file_manager = CoordinateFileManager()

        self.menu_options = {
            1: ("Преобразовать декартовы в сферические", self._cartesian_to_spherical),
            2: ("Преобразовать сферические в декартовы", self._spherical_to_cartesian),
            3: ("Записать координаты в файл", self._write_to_file),
            4: ("Считать координаты из файла", self._read_from_file),
            5: ("Выход", None)
        }

    def _get_float(self, prompt: str) -> float:
        while True:
            try:
                return float(input(prompt))
            except ValueError:
                print("Ошибка: введите корректное число")

    def _cartesian_to_spherical(self):
        x = self._get_float("Введите x: ")
        y = self._get_float("Введите y: ")
        z = self._get_float("Введите z: ")
        self.cartesian = CartesianCoordinates(x, y, z)
        sph = self.cartesian.to_spherical()
        choice = input("Преобразовать углы в градусы? (y/n): ").lower()
        if choice in ["y", "yes", "да"]:
            sph.theta = AngleConverter.rad_to_deg(sph.theta)
            sph.phi = AngleConverter.rad_to_deg(sph.phi)
        self.spherical = sph
        print(f"Сферические координаты: {self.spherical}")

    def _spherical_to_cartesian(self):
        r = self._get_float("Введите r: ")
        theta = self._get_float("Введите theta: ")
        phi = self._get_float("Введите phi: ")
        choice = input("Углы заданы в радианах? (y/n): ").lower()
        if choice in ["n", "no", "нет"]:
            theta = AngleConverter.deg_to_rad(theta)
            phi = AngleConverter.deg_to_rad(phi)
        self.spherical = SphericalCoordinates(r, theta, phi)
        cart = self.spherical.to_cartesian()
        self.cartesian = cart
        print(f"Декартовы координаты: {self.cartesian}")

    def _write_to_file(self):
        filename = input("Введите имя файла: ")
        choice = input("Сохранить (c) декартовы или (s) сферические координаты? ").lower()
        if choice == "c":
            data = [[self.cartesian.x, self.cartesian.y, self.cartesian.z]]
        else:
            data = [[self.spherical.r, self.spherical.theta, self.spherical.phi]]
        self.file_manager.write_to_file(filename, data)
        print(f"Данные успешно записаны в {filename}")

    def _read_from_file(self):
        filename = input("Введите имя файла: ")
        try:
            data = self.file_manager.read_from_file(filename)
            print(f"Считанные данные: {data}")
        except FileNotFoundError:
            print("Ошибка: файл не найден")
        except Exception as e:
            print(f"Ошибка чтения файла: {e}")

    def show(self):
        while True:
            print("\nТекущие координаты:")
            print(f"  Декартовы: {self.cartesian}")
            print(f"  Сферические: {self.spherical}")
            print("\nВыберите действие:")
            for key, (desc, _) in self.menu_options.items():
                print(f"{key}. {desc}")

            try:
                choice = int(input("Введите номер (1-5): "))
                if choice == 5:
                    print("Выход из программы.")
                    break
                elif choice in self.menu_options:
                    self.menu_options[choice][1]()
                else:
                    print("Неверный выбор. Введите число от 1 до 5.")
            except ValueError:
                print("Ошибка: введите число.")
            except Exception as e:
                print(f"Произошла ошибка: {e}")


def show():
    app = GeoTransformApp()
    app.show()


if __name__ == "__main__":
    show()


Overwriting geo_transform/__main__.py


### **Содержимое модуля main.py (с импортом пакета и тестированием функций из него):**

In [11]:
from geo_transform.__main__ import show

show()


Текущие координаты:
CartesianCoordinates(x=0, y=0, z=0)
SphericalCoordinates(r=0, theta=0, phi=0)

Выберите действие:
1. Преобразовать декартовы в сферические
2. Преобразовать сферические в декартовы
3. Записать координаты в файл
4. Считать координаты из файла
5. Выйти
Введите номер действия: 1
Введите x: 1
Введите y: 2
Введите z: 3
Преобразовать в градусы? (y/n): lf
Сферические координаты: SphericalCoordinates(r=3.7416573867739413, theta=1.1071487177940904, phi=0.6405223126794246)

Текущие координаты:
CartesianCoordinates(x=1.0, y=2.0, z=3.0)
SphericalCoordinates(r=3.7416573867739413, theta=1.1071487177940904, phi=0.6405223126794246)

Выберите действие:
1. Преобразовать декартовы в сферические
2. Преобразовать сферические в декартовы
3. Записать координаты в файл
4. Считать координаты из файла
5. Выйти
Введите номер действия: 3
Введите имя файла: test.txt
Какие координаты записать (c/s)? (c - Cartesian, s - Spherical): s
Данные записаны в test.txt

Текущие координаты:
CartesianCoordi